In [ ]:
import numpy as np
import _pickle as pickle
import math
from scipy import stats
from collections import Counter
from PMCA import *
import matplotlib.pyplot as plt
import random

In [ ]:
pid_claim_map = pickle.load(open('./pid_claim_map','rb'))
claim_info_map = pickle.load(open('./claim_info_map','rb'))

In [ ]:
group_id = pickle.load(open('../table/group_id','rb'))
nonzero_index = pickle.load(open('../table/nonzero_index','rb'))
zero_index = []
for i in range(len(pid_claim_map)):
    if i not in nonzero_index: zero_index.append(i)

In [ ]:
def check_icd10_flag(diag_set):
    count_e = 0
    count_v = 0
    icd_10 = ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',\
             'P', 'Q', 'R', 'S', 'T', 'U', 'W', 'X', 'Y', 'Z']
    icd_9 = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    e_code = ['E']
    v_code = ['V']
    for c in diag_set:
        if c[0] in icd_10:return 1
        if c[0] in icd_9:return 0
        if c[0] in e_code:count_e+=1
        if c[0] in v_code:count_v+=1
    if count_e==1 and count_v==0:return 1
    if count_e==0 and count_v==1:return 0
    return 0

In [ ]:
def PC_check(pid_claim_map, claim_info_map, early_age, PMCA_more_flag):
    patient_conditions = []
    for p in pid_claim_map:
        patient_claims = []
        patient_claims_flags = []
        for claim in pid_claim_map[p]:
            claim_codes = []
            for line_trans in claim_info_map[claim]:
                claim_id, diag_set, proc, majcat, svcdt, gender, dob, paid, enc_id = line_trans
                if early_age == 1:
                    if (svcdt - dob).days > 2*365 or (svcdt - dob).days < 0.5*365:continue 
                if early_age == 0:
                    if (svcdt - dob).days > 5*365 or (svcdt - dob).days < 2*365:continue 
                codes = [x.replace('.','') for x in diag_set if x!='']
                claim_codes.extend(codes)
            claim_codes = list(set(claim_codes))
            if svcdt.year >2015: icd10_flag=1
            elif svcdt.year <2015: icd10_flag=0
            else: icd10_flag= check_icd10_flag(claim_codes)
            patient_claims.append(claim_codes)
            patient_claims_flags.append(icd10_flag)
        if PMCA_more_flag==0:patient_conditions.append(PMCA_less(patient_claims, patient_claims_flags))
        if PMCA_more_flag==1:patient_conditions.append(PMCA_more(patient_claims, patient_claims_flags))
    return patient_conditions

In [ ]:
patient_condition_early_more = PC_check(pid_claim_map, claim_info_map, early_age=1, PMCA_more_flag=1)
patient_condition_late_more = PC_check(pid_claim_map, claim_info_map, early_age=0, PMCA_more_flag=1)

patient_condition_early_less = PC_check(pid_claim_map, claim_info_map, early_age=1, PMCA_more_flag=0)
patient_condition_late_less = PC_check(pid_claim_map, claim_info_map, early_age=0, PMCA_more_flag=0)

# Count condition based on group-ID (clusted by K-means)

# Bar plot result

In [ ]:
def bar_list(values, names):
    count = Counter(values)
    bar = [count[i]/len(values) for i in names]
    return bar

In [ ]:
def analysis_plot(patient_condition, zero_index, group_id):
    barWidth = 0.9
    patient_condition = np.array(patient_condition)
    names = ['1 Non-Chronic', '2 Non-complex Chronic', '3 Complex Chronic']

    bar_overall = bar_list(patient_condition, names)
    bar_group_0 = bar_list(patient_condition[zero_index], names)
    bar_group_1 = bar_list(patient_condition[np.array(nonzero_index)[group_id[0]]], names)
    bar_group_2 = bar_list(patient_condition[np.array(nonzero_index)[group_id[1]]], names)
    bar_group_3 = bar_list(patient_condition[np.array(nonzero_index)[group_id[2]]], names)
    bar_group_4 = bar_list(patient_condition[np.array(nonzero_index)[group_id[3]]], names)
    gap = [0]*len(names)
    
    r1 = [1+ x*7 for x in range(len(names))]
    r2 = [2+ x*7 for x in range(len(names))]
    r3 = [3+ x*7 for x in range(len(names))]
    r4 = [4+ x*7 for x in range(len(names))]
    r5 = [5+ x*7 for x in range(len(names))]
    r6 = [6+ x*7 for x in range(len(names))]
    r7 = [7+ x*7 for x in range(len(names))]
    
    plt.xticks([3 + x*7 for x in range(len(names))], [x[2:15] for x in names])

    plt.bar(r1, bar_overall, width = barWidth, color = np.random.rand(3,), label='bar_overall')
    plt.bar(r2, bar_group_0, width = barWidth, color = np.random.rand(3,), label='bar_group_0')
    plt.bar(r3, bar_group_1, width = barWidth, color = np.random.rand(3,), label='bar_group_1')
    plt.bar(r4, bar_group_2, width = barWidth, color = np.random.rand(3,), label='bar_group_2')
    plt.bar(r5, bar_group_3, width = barWidth, color = np.random.rand(3,), label='bar_group_3')
    plt.bar(r6, bar_group_4, width = barWidth, color = np.random.rand(3,), label='bar_group_4')
    plt.bar(r7, gap, width = barWidth)


    plt.legend()
    plt.show()

In [ ]:
analysis_plot(patient_condition_early_more, zero_index, group_id)

In [ ]:
analysis_plot(patient_condition_early_less, zero_index, group_id)

In [ ]:
analysis_plot(patient_condition_late_less, zero_index, group_id)

In [ ]:
analysis_plot(patient_condition_early_less, zero_index, group_id)

# Table result

In [ ]:
def analysis(patient_condition, zero_index, group_id):
    patient_condition = np.array(patient_condition)
    count = Counter(patient_condition)
    print("Overall:")
    for i in sorted(count):
        print(i, ': %i(%0.1f' %(count[i],(count[i] / len(patient_condition)) * 100.0) + '%)')
    print()
    
    count = Counter(patient_condition[zero_index])
    print("For Group-0 (n=%i):"%len(zero_index))
    for i in sorted(count):
        print(i, '%i(%0.1f' %(count[i],(count[i] / len(zero_index)) * 100.0) + '%)')
    print()

    for gid in range(4):
        print("For Group-%i (n=%i):"%(gid+1, len(group_id[gid])))
        count = Counter(patient_condition[np.array(nonzero_index)[group_id[gid]]])
        for i in sorted(count):
            print(i, '%i(%0.1f' %(count[i],(count[i] / len(group_id[gid])) * 100.0) + '%)')
        print()

In [ ]:
print("patient_condition_early_more")
print()
analysis(patient_condition_early_more, zero_index, group_id)

In [ ]:
print("patient_condition_late_more")
print()
analysis(patient_condition_late_more, zero_index, group_id)

In [ ]:
print("patient_condition_early_less")
print()
analysis(patient_condition_early_less, zero_index, group_id)

In [ ]:
print("patient_condition_late_less")
print()
analysis(patient_condition_late_less, zero_index, group_id)